In [1]:
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

print('Success')

Success


In [2]:
table = pd.read_csv('data/Tweets_cleaned.csv')

table

,Toxicity,tweet,filtered_words
0,0,user when a father is dysfunctional and is so ...,"['father', 'dysfunctional', 'selfish', 'drags'..."
1,0,user user thanks for lyft credit i cant use ca...,"['thanks', 'lyft', 'credit', 'cant', 'use', 'c..."
2,0,bihday your majesty,"['bihday', 'majesty']"
3,0,model i love u take with u all the time in ur,"['model', 'love', 'u', 'take', 'u', 'time', 'ur']"
4,0,factsguide society now motivation,"['factsguide', 'society', 'motivation']"
...,...,...,...
56740,1,yous a muthafin lie LifeAsKing Pearls coreyema...,"['yous', 'muthafin', 'lie', 'LifeAsKing', 'Pea..."
56741,1,youve gone and broke the wrong heart baby and ...,"['youve', 'gone', 'broke', 'wrong', 'heart', '..."
56742,1,young buck wanna eat dat nigguh like I aint fu...,"['young', 'buck', 'wanna', 'eat', 'dat', 'nigg..."
56743,1,youu got wild bitches tellin you lies,"['youu', 'got', 'wild', 'bitches', 'tellin', '..."


In [3]:
X = table['filtered_words']
y = table['Toxicity']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [5]:
X_test = X_test.reset_index()

Model from https://huggingface.co/JungleLee/bert-toxic-comment-classification

In [6]:
model_path = "JungleLee/bert-toxic-comment-classification"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

In [7]:
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# print(pipeline("You're a fucking nerd."))

In [8]:
y_pred = []

In [9]:
for i in range(len(X_test)):
    y_pred.append(pipeline(str(X_test['filtered_words'][i])))

In [10]:
for i, item in enumerate(y_pred):
    if isinstance(item, dict):
        y_pred[i] = 1 if item['label'] == 'toxic' else 0
    elif isinstance(item, list):
        for j, subitem in enumerate(item):
            if isinstance(subitem, dict):
                item[j] = 1 if subitem['label'] == 'toxic' else 0

In [11]:
def flatten(lst):
    flat_list = []
    for item in lst:
        if isinstance(item, list):
            flat_list.extend(flatten(item))
        else:
            flat_list.append(item)
    return flat_list

In [12]:
y_pred = flatten(y_pred)

In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.97      0.83      6472
           1       0.94      0.51      0.66      4877

    accuracy                           0.78     11349
   macro avg       0.83      0.74      0.75     11349
weighted avg       0.82      0.78      0.76     11349



In [14]:
print('f1_score = ', f1_score(y_test, y_pred))

f1_score =  0.6616262103727285
